In [23]:
from datetime import datetime
start_time = datetime.now()
import os
import re
import nltk
import pandas as pd
import got3
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import FrenchStemmer
from nltk import NaiveBayesClassifier
from sklearn.model_selection import train_test_split

## Importing and cleaning train 

In [24]:
train= pd.read_csv('./nlp/tweets.csv', sep='^([^,]+),', engine='python', error_bad_lines=False, encoding='utf-8', index_col=[0])

In [25]:
train.head()

,polarity,statutnull
NaN,0,"- Awww, c'est nul. Tu aurais du prendre David ..."
NaN,0,Est contrarié qu'il ne puisse pas mettre à jou...
NaN,0,J'ai plongé plusieurs fois pour la balle. A ré...
NaN,0,Tout mon corps a des démangeaisons et comme si...
NaN,0,"Non, il ne se comporte pas bien du tout. je su..."


In [26]:
train = train.reset_index(drop=True)

In [27]:
train.head()

,polarity,statutnull
0,0,"- Awww, c'est nul. Tu aurais du prendre David ..."
1,0,Est contrarié qu'il ne puisse pas mettre à jou...
2,0,J'ai plongé plusieurs fois pour la balle. A ré...
3,0,Tout mon corps a des démangeaisons et comme si...
4,0,"Non, il ne se comporte pas bien du tout. je su..."


In [28]:
train.columns

Index(['polarity', 'statutnull'], dtype='object')

In [29]:
train['polarity'] = train['polarity'].str.replace("0","negatif").str.replace("4","positif")

In [30]:
index_zero = train[(train['polarity'] != 'positif') & (train['polarity']!= 'negatif')].index

In [31]:
train.drop(index_zero, inplace=True)

In [32]:
train = train.dropna()

In [33]:
train = train.sample(n=200000)

In [34]:
train['polarity'].value_counts()
#.plot(kind='bar')

positif    100400
negatif     99600
Name: polarity, dtype: int64

In [35]:
set(train['polarity'])

{'negatif', 'positif'}

In [36]:
def clean_up(tweet):
    tweet = re.sub(r'http\S+|(pic.twitter\.[^\s]+)|(www\.[^\s]+)|(@\S+)|\s\s+|[^\w\s]',' ',tweet) 
    #1:http+suite/2.pic.twitter+suite/3.www.+suite/4.@+suite/5.espaces++/6.ponctuation
    tweet = tweet.lower().strip() #bdc
    tweet = word_tokenize(tweet) #tokenisation
    stop_words = stopwords.words('french') #stopwords nltk
    stop_words.append('rt') #+'rt'
    tweet = [word for word in tweet if word not in stop_words]
    tweet = [word for word in tweet if len(word)>1] #exclus mot de 1 lettre
    return ' '.join(tweet) #retour sans tokenisation, requis par spacy
def stem_spacy(tweet):
    import spacy
    nlp = spacy.load('fr_core_news_sm')
    tweet = tweet.apply(nlp)
    tweet_stem=[]
    for doc in tweet:
        tweet_stem.append([word.lemma_ for word in doc])
    return tweet_stem

In [37]:
train.statutnull = train.statutnull.astype(str)

In [38]:
train['original'] = train['statutnull']

In [39]:
train.statutnull = train.statutnull.apply(lambda s: clean_up(s))

In [40]:
train.head()

,polarity,statutnull,original
699617,negatif,coincé travail alors 160 haralson détruit croi...,Je suis coincé au travail alors que 160 harals...
1392903,positif,droit queue,Droit sur la queue
165893,negatif,travail nul,Mon travail est nul !!!
406813,negatif,argh trouvais logistique aller où courir lorsq...,Argh. Je trouvais la logistique d'aller et d'o...
1085748,positif,avoir belle journée famille fait bien,Avoir une belle journée avec la famille. sa fa...


In [41]:
train.statutnull = stem_spacy(train.statutnull)

In [42]:
train.head()

,polarity,statutnull,original
699617,negatif,"[coincer, travail, alors, 160, haralson, détru...",Je suis coincé au travail alors que 160 harals...
1392903,positif,"[droit, queue]",Droit sur la queue
165893,negatif,"[travail, nul]",Mon travail est nul !!!
406813,negatif,"[argh, trouver, logistique, aller, où, courir,...",Argh. Je trouvais la logistique d'aller et d'o...
1085748,positif,"[avoir, beau, journée, famille, faire, bien]",Avoir une belle journée avec la famille. sa fa...


In [43]:
train.rename(columns={'statutnull':'tweet'},inplace=True)

In [44]:
train.shape

(200000, 3)

In [45]:
train.reset_index(drop=True).head()

,polarity,tweet,original
0,negatif,"[coincer, travail, alors, 160, haralson, détru...",Je suis coincé au travail alors que 160 harals...
1,positif,"[droit, queue]",Droit sur la queue
2,negatif,"[travail, nul]",Mon travail est nul !!!
3,negatif,"[argh, trouver, logistique, aller, où, courir,...",Argh. Je trouvais la logistique d'aller et d'o...
4,positif,"[avoir, beau, journée, famille, faire, bien]",Avoir une belle journée avec la famille. sa fa...


In [46]:
train['tweet_processed2'] = train.tweet.apply(lambda x: ' '.join(x))
train.head()

,polarity,tweet,original,tweet_processed2
699617,negatif,"[coincer, travail, alors, 160, haralson, détru...",Je suis coincé au travail alors que 160 harals...,coincer travail alors 160 haralson détruire cr...
1392903,positif,"[droit, queue]",Droit sur la queue,droit queue
165893,negatif,"[travail, nul]",Mon travail est nul !!!,travail nul
406813,negatif,"[argh, trouver, logistique, aller, où, courir,...",Argh. Je trouvais la logistique d'aller et d'o...,argh trouver logistique aller où courir lorsqu...
1085748,positif,"[avoir, beau, journée, famille, faire, bien]",Avoir une belle journée avec la famille. sa fa...,avoir beau journée famille faire bien


## Building model 

In [ ]:
all_words = []
NUM_FEATURES = 5000
for index, value in train.tweet.iteritems():
    if value not in all_words:
        all_words += value
top_features = [x[0] for x in nltk.FreqDist(all_words).most_common(NUM_FEATURES)]

In [ ]:
df=pd.DataFrame.from_dict(nltk.FreqDist(all_words),orient='index')
df.sort_values(by=0, ascending=False).reset_index().head(10)

In [ ]:
def build_features(words):
    features = {}
    for w in top_features:
        features[w] = (w in words)
    return features

In [ ]:
featuresets = []

for index, row in train.iterrows():
    featuresets.append((build_features(row['tweet']), row['polarity']))

In [ ]:
featuresets[:2]

In [ ]:
train_set, test = train_test_split(featuresets, test_size=0.2)
classifier = NaiveBayesClassifier.train(train_set)

In [ ]:
classifier.show_most_informative_features(n=10)

In [ ]:
nltk.classify.accuracy(classifier, test)
# score 73,56% pour sample 100.000 avec stem nltk
# score 73,88% avec sample 100.000 avec stem spacy 

In [ ]:
phrase=['détester','grève']
#phrase_clean=clean_up(phrase)
#phrase_clean=stem_spacy(phrase_clean)
#phrase_clean

In [ ]:
classifier.classify(dict([token,True] for token in phrase))

In [ ]:
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

In [ ]:
from datetime import datetime
start_time = datetime.now()
import got3

In [ ]:
#https://github.com/Jefferson-Henrique/GetOldTweets-python
def tweetSearch(query, date_since,max_tweets):
    tweetCriteria = got3.manager.TweetCriteria().setSince(date_since).setQuerySearch(query).setMaxTweets(max_tweets).setLang('french')
    text=""
    date=""
    for i in range(max_tweets):
        tweet = got3.manager.TweetManager.getTweets(tweetCriteria)[i]
        text+= tweet.text.lower()+";;"
        date+= str(tweet.date)+";;"
    return text,date

In [ ]:
tweets = tweetSearch("#Grève","2019-12-05",500)

In [ ]:
greve = tweets[0].split(';;')
date = tweets[1].split(';;')

In [ ]:
greve_twitter = pd.DataFrame(columns=['polarity','date','tweet'])

In [ ]:
greve_twitter['tweet']=greve
greve_twitter['date']=date

In [ ]:
greve_twitter['tweet_processed'] = greve_twitter['tweet'].astype(str).apply(lambda s: clean_up(s))

In [ ]:
greve_twitter['tweet_processed2'] = stem_spacy(greve_twitter['tweet_processed'])

In [ ]:
for i in range(len(greve_twitter)):
    greve_twitter['polarity'].iloc[i] = classifier.classify(dict([token,True] for token in greve_twitter['tweet_processed2'].iloc[i]))

In [ ]:
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

In [ ]:
greve_twitter.to_csv('greve_twitter_final.csv')

In [ ]:
greve_twitter.sample(n=10)